In [14]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from graphframes import GraphFrame

import findspark
findspark.init()


python_path = r'C:\Users\Sheri\AppData\Local\Programs\Python\Python311\python.exe'  
os.environ['PYSPARK_PYTHON'] = python_path
os.environ['PYSPARK_DRIVER_PYTHON'] = python_path

spark = SparkSession.builder \
    .appName("PySpark Installation Test") \
    .getOrCreate()

# Define vertices (users, artists, tags)
users = spark.createDataFrame([
    (1, "User1"),
    (2, "User2")
], ["id", "name"])

artists = spark.createDataFrame([
    (3, "Artist1"),
    (4, "Artist2")
], ["id", "name"])

tags = spark.createDataFrame([
    (5, "Tag1"),
    (6, "Tag2")
], ["id", "name"])

# Define edges (User-Artist, Artist-Tag)
userArtistEdges = spark.createDataFrame([
    (1, 3, "listens"),
    (2, 4, "listens")
], ["src", "dst", "relationship"])

artistTagEdges = spark.createDataFrame([
    (3, 5, "tagged"),
    (4, 6, "tagged")
], ["src", "dst", "relationship"])

# Combine all vertices and edges
vertices = users.union(artists).union(tags)
edges = userArtistEdges.union(artistTagEdges)

# Create the graph
graph = GraphFrame(vertices, edges)

# Show vertices and edges
graph.vertices.show()
graph.edges.show()

+---+-------+
| id|   name|
+---+-------+
|  1|  User1|
|  2|  User2|
|  3|Artist1|
|  4|Artist2|
|  5|   Tag1|
|  6|   Tag2|
+---+-------+

+---+---+------------+
|src|dst|relationship|
+---+---+------------+
|  1|  3|     listens|
|  2|  4|     listens|
|  3|  5|      tagged|
|  4|  6|      tagged|
+---+---+------------+

